# GDAL - interacting with WMS

GDAL - Geospatial Data Abstration Library

#### Set up
Use - `opengeo` micromamba env. 
Move to `gdal-tools` folder.

In [1]:
cd /Users/abharathi/Documents/gis_data/gdal-tools/

/Users/abharathi/Documents/gis_data/gdal-tools


In [2]:
ls

1870_southern-india.jpg*      landsat8/
5d16a93f1cf0f6000579ad2c.tif  london_1m_dsm/
batch.py*                     naip/
batch_parallel.py*            precipitation.gpkg
earth_at_night.jpg*           prism/
earthquakes/                  spatial_query.gpkg
gdal_stuff.qgz                srtm/
geonames/                     worldcities.csv*


## WMS with GDAL
You can download tiles. You can use `gdal_translate -of WMS <url> output.xml` to create a layer file in XML that you can share the url to consumers of qgis.

### List layers

In [3]:
!gdalinfo "WMS:https://sedac.ciesin.columbia.edu/geoserver/ows?version=1.3.0"

Driver: WMS/OGC Web Map Service
Files: none associated
Size is 512, 512
Subdatasets:
  SUBDATASET_1_NAME=WMS:https://sedac.ciesin.columbia.edu/geoserver/ows?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=cartographic%3Anational-boundaries&CRS=EPSG:4326&BBOX=-55.792,-180.0,83.667,180.0
  SUBDATASET_1_DESC=cartographic:national-boundaries
  SUBDATASET_2_NAME=WMS:https://sedac.ciesin.columbia.edu/geoserver/ows?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=cartographic%3A00-global-labels&CRS=CRS:84&BBOX=-177.225,-51.741,178.45,78.877
  SUBDATASET_2_DESC=Global Labels
  SUBDATASET_3_NAME=WMS:https://sedac.ciesin.columbia.edu/geoserver/ows?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=cartographic%3A00-gpw-v3-national-admin-boundaries&CRS=CRS:84&BBOX=-180.0,-55.792,180.0,83.667
  SUBDATASET_3_DESC=National Boundaries Medium Scale
  SUBDATASET_4_NAME=WMS:https://sedac.ciesin.columbia.edu/geoserver/ows?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=cartographic%3A00-gpw-v3-national-admin

In [4]:
!gdalinfo "WMS:https://sedac.ciesin.columbia.edu/geoserver/ows?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=grand-v1%3Agrand-v1-reservoirs-rev01&CRS=CRS:84&BBOX=-153.037,-45.881,176.825,70.398"

Driver: WMS/OGC Web Map Service
Files: none associated
Size is 1073741824, 378502602
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ID["EPSG",6326]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433],
        ID["EPSG",8901]],
    CS[ellipsoidal,2],
        AXIS["longitude",east,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433,
                ID["EPSG",9122]]],
        AXIS["latitude",north,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433,
                ID["EPSG",9122]]]]
Data axis to CRS axis mapping: 1,2
Origin = (-153.037000000000006,70.397999999999996)
Pixel Size = (0.000000307207927,-0.000000307207928)
Image Structure Metadata:
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  (-153.0370000,  70.3980000) (153d 2'13.20"W, 70d23'52.80"N)
Lower Left  (-153.0370000, -45.8810000) (153d 2

In [5]:
!gdal_translate -of WMS "WMS:https://sedac.ciesin.columbia.edu/geoserver/ows?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&LAYERS=grand-v1%3Agrand-v1-reservoirs-rev01&CRS=CRS:84&BBOX=-153.037,-45.881,176.825,70.398" reservoirs.xml

Input file size is 1073741824, 378502602


### Download WMS as rasters

In [7]:
%%time
!gdalwarp -tr 0.005 0.005 -te 68.106 6.762 97.412 37.078 reservoirs.xml \
  reservoirs_india.tif -co COMPRESS=DEFLATE -co TILED=YES

Creating output file that is 5861P x 6063L.
Processing reservoirs.xml [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 333 ms, sys: 111 ms, total: 444 ms
Wall time: 22.3 s


#### Specify desired extent in an easy to use SRS

In [9]:
%%time
!gdalwarp -tr 500 500 -te 68.106 6.762 97.412 37.078 -te_srs EPSG:4326 \
  reservoirs.xml \
  -t_srs EPSG:7755 reservoirs_india_proj.tif -co COMPRESS=DEFLATE -co TILED=YES

Creating output file that is 5803P x 6744L.
Processing reservoirs.xml [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 125 ms, sys: 48.1 ms, total: 173 ms
Wall time: 8.55 s


## Georeferencing

### With corner coords

In [10]:
ls earth*

earth_at_night.jpg*

earthquakes:
2020_01.geojson* 2020_04.geojson* 2020_07.geojson* 2020_10.geojson*
2020_02.geojson* 2020_05.geojson* 2020_08.geojson* 2020_11.geojson*
2020_03.geojson* 2020_06.geojson* 2020_09.geojson* 2020_12.geojson*


In [11]:
!gdalinfo earth_at_night.jpg

Driver: JPEG/JPEG JFIF
Files: earth_at_night.jpg
Size is 13500, 6750
Image Structure Metadata:
  COMPRESSION=JPEG
  INTERLEAVE=PIXEL
  SOURCE_COLOR_SPACE=YCbCr
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0, 6750.0)
Upper Right (13500.0,    0.0)
Lower Right (13500.0, 6750.0)
Center      ( 6750.0, 3375.0)
Band 1 Block=13500x1 Type=Byte, ColorInterp=Red
  Overviews: 6750x3375, 3375x1688, 1688x844
  Image Structure Metadata:
    COMPRESSION=JPEG
Band 2 Block=13500x1 Type=Byte, ColorInterp=Green
  Overviews: 6750x3375, 3375x1688, 1688x844
  Image Structure Metadata:
    COMPRESSION=JPEG
Band 3 Block=13500x1 Type=Byte, ColorInterp=Blue
  Overviews: 6750x3375, 3375x1688, 1688x844
  Image Structure Metadata:
    COMPRESSION=JPEG


In [12]:
# use -a_ullr for upper left, lower right coords
!gdal_translate -a_ullr -180 90 180 -90 -a_srs EPSG:4326 \
  earth_at_night.jpg earth_at_night.tif \
  -co COMPRESS=JPEG -co TILED=YES -co PHOTOMETRIC=RGB

Input file size is 13500, 6750
0...10...20...30...40...50...60...70...80...90...100 - done.


### With GCP points
GeoTiff can store GCP in header, and can apply the transformation on the fly. We then use gdalwarp to apply the data and persist to disk.

In [13]:
!gdal_translate -gcp 418 893 70 15 -gcp 380 2432 70 5 -gcp 3453 2434  90 5 \
  -gcp 3407 895 90 15 -gcp 2662 911 85 15 \
  1870_southern-india.jpg india-with-gcp.vrt

Input file size is 3957, 3071


In [14]:
!gdalwarp -t_srs EPSG:4042 -order 1 -tr 0.005 0.005 \
  india-with-gcp.vrt india-reprojected-polynomial.tif \
  -co COMPRESS=JPEG -co JPEG_QUALITY=50 -co PHOTOMETRIC=YCBCR

Creating output file that is 5241P x 4012L.
Processing india-with-gcp.vrt [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


#### Project using a different warping algorithm

In [17]:
!gdalwarp -t_srs EPSG:4042 -tr 0.005 0.005 \
  india-with-gcp.vrt india-reprojected-tps.tif \
  -co COMPRESS=JPEG -co JPEG_QUALITY=50 -co PHOTOMETRIC=YCBCR

Creating output file that is 5241P x 4012L.
Processing india-with-gcp.vrt [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


### Idea - georeference and animate weather data?